In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('scraping-382202-fa58f9c946e5.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#
date = 0 #当日は0、前日は1・・・
#date = 1 #当日は0、前日は1・・・
date2 = "08/19"
pmc = 28013002 # 店舗ID
SPREADSHEET_KEY = '1C-OecHzGLBXw7BqNcmKmVE24YhivoFnWBqdsSrN4UGk' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 601 # スロットの1台目
wait = 0.02

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
path = "C:\\Users\\garden026\\WD\\" + str(wd) + "\\chromedriver.exe"

chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
#s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
#s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

time.sleep(5)

SPREADSHEET_KEY = '1d9OZcjkMzQFIPdWuomH3J4J_xizX8or_X_xbem4P_gc' # スプレッドシートのID
gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
s5row = worksheet.cell(11,  29).value

time.sleep(5)

wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025689', '1', '21282.0', '-10531.0', '13', '6']
['08/19', '025521', '2', '7949.0', '-1203.0', '7', '5']
['08/19', '025460', '3', '5641.0', '1506.0', '7', '1']
['08/19', '025544', '4', '10769.0', '-3309.0', '19', '7']
['08/19', '025892', '5', '26154.0', '-8726.0', '18', '10']
['08/19', '025865', '6', '13846.0', '5719.0', '59', '5']
['08/19', '025891', '7', '36154.0', '18959.0', '80', '26']
['08/19', '025689', '8', '13077.0', '-1504.0', '15', '5']
['08/19', '025841', '9', '12821.0', '-9629.0', '2', '2']
['08/19', '025835', '10', '14103.0', '603.0', '50', '6']
['08/19', '025741', '11', '15128.0', '-5416.0', '9', '5']
['08/19', '025613', '12', '20000.0', '-10531.0', '10', '7']
['08/19', '025771', '13', '16923.0', '-5717.0', '30', '10']
['08/19', '025533', '14', '22821.0', '-2707.0', '21', '7']
['08/19', '025891', '15', '19744.0', '-5115.0', '46', '13']
['08/19', '025855', '16', '5128.0', '603.0', '11', '2']
['08/19', '025892', '17', '26154.0', '-1504.0', '30', '7']
['08/19', '02

['08/19', '025489', '141', '37949.0', '6320.0', '19', '4']
['08/19', '025489', '142', '26923.0', '-9629.0', '6', '3']
['08/19', '025489', '143', '43590.0', '-15045.0', '10', '5']
['08/19', '025489', '144', '28205.0', '-9027.0', '7', '4']
['08/19', '025489', '145', '42564.0', '-1203.0', '17', '7']
['08/19', '025489', '146', '24103.0', '22570.0', '24', '3']
['08/19', '025489', '147', '38205.0', '15950.0', '25', '7']
['08/19', '025489', '148', '15641.0', '4515.0', '9', '3']
['08/19', '025489', '149', '52308.0', '603.0', '21', '9']
['08/19', '025489', '150', '38718.0', '6621.0', '19', '5']
['08/19', '025489', '151', '40000.0', '-18957.0', '6', '4']
['08/19', '025489', '152', '32564.0', '2408.0', '15', '5']
['08/19', '025489', '153', '43590.0', '-3610.0', '17', '7']
['08/19', '025489', '154', '33846.0', '-15346.0', '5', '3']
['08/19', '025489', '155', '35385.0', '4515.0', '17', '7']
['08/19', '025489', '156', '37436.0', '-3610.0', '14', '6']
['08/19', '025489', '157', '21026.0', '4214.0', '

['08/19', '025317', '282', '37436.0', '1806.0', '46', '12']
['08/19', '025765', '283', '43077.0', '-3309.0', '45', '18']
['08/19', '025521', '284', '39487.0', '3010.0', '46', '20']
['08/19', '025605', '285', '43846.0', '-6619.0', '42', '17']
['08/19', '025689', '286', '31282.0', '-7221.0', '28', '10']
['08/19', '025521', '287', '34359.0', '1806.0', '38', '16']
['08/19', '025735', '288', '32051.0', '1806.0', '30', '14']
['08/19', '025533', '289', '38462.0', '9931.0', '59', '14']
['08/19', '025523', '290', '23333.0', '12640.0', '47', '12']
['08/19', '025285', '291', '36154.0', '6621.0', '41', '12']
['08/19', '025697', '292', '34359.0', '603.0', '56', '19']
['08/19', '025409', '293', '26923.0', '-2406.0', '57', '9']
['08/19', '025287', '294', '33333.0', '-6319.0', '54', '9']
['08/19', '025605', '295', '27949.0', '-9328.0', '17', '8']
['08/19', '025521', '296', '42308.0', '-4814.0', '39', '18']
['08/19', '025364', '297', '31795.0', '8126.0', '49', '15']
['08/19', '025655', '298', '31282.0'

['08/19', '025844', '421', '8718.0', '-6619.0', '3', '3']
['08/19', '025844', '422', '3846.0', '-3309.0', '1', '1']
['08/19', '025844', '423', '3590.0', '-3008.0', '1', '1']
['08/19', '025844', '424', '6154.0', '5719.0', '12', '2']
['08/19', '025844', '425', '17436.0', '-14143.0', '4', '4']
['08/19', '025844', '426', '2051.0', '-2106.0', '1', '1']
['08/19', '025876', '427', '4359.0', '-3610.0', '1', '1']
['08/19', '025876', '428', '4615.0', '-902.0', '3', '1']
['08/19', '025876', '429', '4872.0', '-3911.0', '1', '1']
['08/19', '025876', '430', '10256.0', '3612.0', '11', '1']
['08/19', '025893', '434', '6154.0', '1806.0', '10', '1']
['08/19', '025893', '435', '14359.0', '-4814.0', '9', '2']
['08/19', '025893', '436', '8462.0', '1205.0', '11', '1']
['08/19', '025900', '437', '38974.0', '51760.0', '54', '9']
['08/19', '025900', '438', '29744.0', '15348.0', '26', '5']
['08/19', '025900', '439', '35897.0', '16552.0', '30', '7']
['08/19', '025900', '440', '42308.0', '-12337.0', '16', '5']
['

['08/19', '119997', '649', '5358', '1991.0', '23', '16', '--']
['08/19', '119997', '650', '3719', '-972.0', '9', '10', '--']
['08/19', '119997', '651', '4481', '185.0', '16', '10', '--']
['08/19', '119997', '652', '4824', '231.0', '16', '15', '--']
['08/19', '119997', '653', '2166', '-324.0', '7', '4', '--']
['08/19', '119997', '654', '3878', '231.0', '13', '11', '--']
['08/19', '120085', '655', '3375', '-648.0', '18', '7', '--']
['08/19', '120085', '656', '2497', '-2361.0', '5', '4', '--']
['08/19', '120085', '657', '2500', '-2037.0', '6', '5', '--']
['08/19', '120085', '658', '2021', '556.0', '13', '8', '--']
['08/19', '120085', '659', '2837', '3426.0', '33', '9', '--']
['08/19', '120085', '660', '3064', '694.0', '20', '12', '--']
['08/19', '120022', '661', '3952', '-1481.0', '20', '11', '--']
['08/19', '120022', '662', '3404', '2454.0', '32', '18', '--']
['08/19', '120022', '663', '1585', '-231.0', '9', '5', '--']
['08/19', '120022', '664', '2551', '-139.0', '16', '8', '--']
['08/19

['08/19', '120102', '781', '3042', '-3426.0', '14', '12', '--']
['08/19', '120104', '782', '2742', '602.0', '13', '5', '--']
['08/19', '119950', '783', '3181', '-370.0', '45', '10', '--']
['08/19', '120121', '784', '4771', '-1944.0', '73', '18', '--']
['08/19', '120115', '785', '2595', '-2037.0', '29', '4', '--']
['08/19', '120034', '786', '3153', '1250.0', '52', '9', '--']
['08/19', '120117', '787', '3898', '3380.0', '99', '24', '--']
['08/19', '119940', '788', '4203', '-2546.0', '17', '14', '--']
['08/19', '119973', '789', '2804', '139.0', '36', '10', '--']
['08/19', '120091', '790', '2504', '1343.0', '45', '15', '--']
['08/19', '120108', '791', '6390', '-3426.0', '26', '0', '--']
['08/19', '120032', '792', '441', '1806.0', '28', '1', '--']


{'spreadsheetId': '1d9OZcjkMzQFIPdWuomH3J4J_xizX8or_X_xbem4P_gc',
 'updates': {'spreadsheetId': '1d9OZcjkMzQFIPdWuomH3J4J_xizX8or_X_xbem4P_gc',
  'updatedRange': 'S5RAW!F36616:M36807',
  'updatedRows': 192,
  'updatedColumns': 8,
  'updatedCells': 1536}}

In [12]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
#s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
#wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

time.sleep(5)

SPREADSHEET_KEY = '1d9OZcjkMzQFIPdWuomH3J4J_xizX8or_X_xbem4P_gc' # スプレッドシートのID
gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
s5row = worksheet.cell(11,  29).value

time.sleep(5)

wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['07/28', '025689', '1', '11026.0', '8427.0', '23', '3']
['07/28', '025755', '2', '0', '0', '2', '1']
['07/28', '025460', '3', '3846.0', '-1805.0', '2', '1']
['07/28', '025392', '4', '4103.0', '1806.0', '17', '1']
['07/28', '025465', '5', '256.0', '1506.0', '4', '1']
['07/28', '025534', '6', '0', '0', '0', '0']
['07/28', '025868', '7', '0', '0', '0', '0']
['07/28', '025689', '8', '3333.0', '-1504.0', '3', '2']
['07/28', '025841', '9', '1795.0', '-902.0', '1', '1']
['07/28', '025835', '10', '10769.0', '-4814.0', '22', '7']
['07/28', '025782', '11', '256.0', '-902.0', '0', '0']
['07/28', '025613', '12', '2821.0', '-2106.0', '1', '1']
['07/28', '025771', '13', '7949.0', '-2406.0', '14', '5']
['07/28', '025533', '14', '9487.0', '904.0', '16', '5']
['07/28', '025601', '15', '0', '0', '0', '0']
['07/28', '025855', '16', '11538.0', '-902.0', '22', '6']
['07/28', '025776', '17', '9231.0', '302.0', '21', '6']
['07/28', '025765', '18', '7179.0', '3913.0', '14', '3']
['07/28', '025863', '19', '51

['07/28', '025489', '144', '3846.0', '-1203.0', '1', '1']
['07/28', '025489', '145', '11026.0', '8126.0', '9', '3']
['07/28', '025489', '146', '21026.0', '904.0', '9', '2']
['07/28', '025489', '147', '6410.0', '-5115.0', '0', '0']
['07/28', '025489', '148', '3590.0', '-902.0', '1', '1']
['07/28', '025489', '149', '13333.0', '2709.0', '7', '2']
['07/28', '025489', '150', '16410.0', '-2406.0', '6', '2']
['07/28', '025489', '151', '30256.0', '5117.0', '15', '5']
['07/28', '025489', '152', '10256.0', '2709.0', '6', '1']
['07/28', '025489', '153', '18205.0', '-6619.0', '4', '3']
['07/28', '025489', '154', '6923.0', '-3610.0', '1', '1']
['07/28', '025489', '155', '769.0', '-1203.0', '0', '0']
['07/28', '025489', '156', '17179.0', '-5717.0', '4', '2']
['07/28', '025489', '157', '4359.0', '-3610.0', '0', '0']
['07/28', '025489', '158', '6154.0', '-4814.0', '0', '0']
['07/28', '025489', '159', '23077.0', '-2406.0', '9', '3']
['07/28', '025489', '160', '27949.0', '4515.0', '15', '3']
['07/28', '

['07/28', '025521', '284', '37949.0', '-902.0', '37', '16']
['07/28', '025605', '285', '13590.0', '-2707.0', '10', '2']
['07/28', '025689', '286', '24103.0', '6320.0', '35', '11']
['07/28', '025521', '287', '48205.0', '-11434.0', '33', '19']
['07/28', '025735', '288', '46923.0', '-14744.0', '24', '16']
['07/28', '025533', '289', '22821.0', '3612.0', '36', '8']
['07/28', '025523', '290', '22308.0', '6019.0', '30', '9']
['07/28', '025285', '291', '19744.0', '1806.0', '24', '11']
['07/28', '025697', '292', '24872.0', '8126.0', '58', '13']
['07/28', '025409', '293', '30769.0', '5117.0', '97', '8']
['07/28', '025287', '294', '20513.0', '-7221.0', '27', '4']
['07/28', '025605', '295', '38205.0', '603.0', '42', '22']
['07/28', '025521', '296', '36154.0', '-4212.0', '30', '15']
['07/28', '025364', '297', '13077.0', '-2406.0', '12', '4']
['07/28', '025655', '298', '18462.0', '603.0', '43', '8']
['07/28', '025404', '299', '37949.0', '9029.0', '53', '16']
['07/28', '025317', '300', '36154.0', '87

['07/28', '025876', '427', '4872.0', '-3610.0', '1', '1']
['07/28', '025876', '428', '0', '0', '0', '0']
['07/28', '025876', '429', '3846.0', '6019.0', '9', '4']
['07/28', '025876', '430', '3846.0', '-3008.0', '0', '0']
['07/28', '025826', '431', '769.0', '-902.0', '0', '0']
['07/28', '025826', '432', '7436.0', '-4814.0', '1', '1']
['07/28', '025826', '433', '3333.0', '1806.0', '3', '1']
['07/28', '025606', '434', '12308.0', '-6920.0', '4', '3']
['07/28', '025606', '435', '6154.0', '-4212.0', '1', '1']
['07/28', '025606', '436', '4359.0', '-3610.0', '0', '0']
['07/28', '025606', '437', '15897.0', '-1805.0', '8', '3']
['07/28', '025606', '438', '3077.0', '-2406.0', '0', '0']
['07/28', '025606', '439', '7949.0', '-5416.0', '1', '1']
['07/28', '025606', '440', '6154.0', '1506.0', '5', '2']
['07/28', '025764', '441', '12051.0', '-10832.0', '0', '0']
['07/28', '025764', '442', '2051.0', '-902.0', '1', '1']
['07/28', '025764', '443', '513.0', '-1203.0', '0', '0']
['07/28', '025764', '444', '

['07/28', '119997', '654', '1428', '-648.0', '2', '4', '--']
['07/28', '120085', '655', '3447', '-787.0', '18', '7', '--']
['07/28', '120085', '656', '1318', '-648.0', '4', '5', '--']
['07/28', '120085', '657', '2837', '-1065.0', '10', '12', '--']
['07/28', '120085', '658', '2224', '3657.0', '28', '12', '--']
['07/28', '120085', '659', '2475', '-2037.0', '11', '6', '--']
['07/28', '120085', '660', '2901', '-694.0', '13', '10', '--']
['07/28', '120022', '661', '2810', '-509.0', '17', '6', '--']
['07/28', '120022', '662', '4637', '-1991.0', '26', '13', '--']
['07/28', '120022', '663', '3408', '-2917.0', '9', '12', '--']
['07/28', '120022', '664', '2612', '-833.0', '13', '8', '--']
['07/28', '120022', '665', '4779', '-1528.0', '24', '18', '--']
['07/28', '120022', '666', '4048', '4583.0', '49', '18', '--']
['07/28', '119962', '667', '3228', '509.0', '11', '11', '--']
['07/28', '119962', '668', '6756', '926.0', '28', '18', '--']
['07/28', '119962', '669', '4802', '-185.0', '15', '13', '--'

['07/28', '120034', '786', '3280', '93.0', '43', '6', '--']
['07/28', '120117', '787', '2725', '-602.0', '57', '12', '--']
['07/28', '119940', '788', '1731', '-185.0', '13', '8', '--']
['07/28', '119973', '789', '5545', '833.0', '67', '22', '--']
['07/28', '120091', '790', '2575', '-2361.0', '16', '8', '--']
['07/28', '120132', '791', '2808', '-1204.0', '11', '7', '--']
['07/28', '120032', '792', '2291', '1065.0', '41', '4', '--']


{'spreadsheetId': '1d9OZcjkMzQFIPdWuomH3J4J_xizX8or_X_xbem4P_gc',
 'updates': {'spreadsheetId': '1d9OZcjkMzQFIPdWuomH3J4J_xizX8or_X_xbem4P_gc',
  'updatedRange': 'S5RAW!F32584:M32775',
  'updatedRows': 192,
  'updatedColumns': 8,
  'updatedCells': 1536}}